## Getting Code and Preparing Executable Programs

Getting the GitHub Codebase

In [ ]:
!wget https://github.com/yanlab19870714/Tthinker/archive/main.zip #download
!unzip main.zip # decompress
!rm main.zip # now that "Tthinker-main" folder is available, remove zip file 

Compiling the programs

In [ ]:
!cd Tthinker-main/app_qc && make clean && make # program: run
!cd Tthinker-main/maximal_check && make clean && make # program: quasiCliques
!cd Tthinker-main/app_kernel && make clean && make # program: run

## Preparing the Test Dataset: Arxiv GR-QC

The dataset is an arXiv collaboration network. More details can be found here: https://snap.stanford.edu/data/ca-GrQc.html

In [ ]:
!wget https://snap.stanford.edu/data/ca-GrQc.txt.gz

Decompress (unzip) the dataset

In [ ]:
!gunzip ca-GrQc.txt.gz # this will give you the file "ca-GrQc.txt"

Take a look at the file

In [ ]:
!head ca-GrQc.txt

The first 4 lines are metadata, remove them so that we get an edge list

In [ ]:
!sed -i'.original' -e '1,4d' ca-GrQc.txt

In [ ]:
!head ca-GrQc.txt

In [ ]:
!rm ca-GrQc.txt.original

Convert the edge format to adjacent list format

In [ ]:
!python3 Tthinker-main/demo/edge2adj.py ca-GrQc.txt # this generates file ca-GrQc.txt_adj

In [ ]:
!head ca-GrQc.txt_adj # vertex_ID  adjacency_list_length \t neigbhor1ID  neigbhor2ID  ...

The quasi-clique input format should be one vertex per line, and each line only needs the vertex IDs in the adjacency list.

Right now the vertices have been sorted by ID, however, there are some missing vertices like 15 and 23 which we need to insert empty lines.

We also need to remove the first 2 columns which are not part of the adjacency lists. The output file is named as `input_data`.

In [ ]:
!cp Tthinker-main/demo/prepare_quick.sh .
!cp Tthinker-main/demo/add_null.py .
!chmod +x prepare_quick.sh
!./prepare_quick.sh ca-GrQc.txt_adj input_data

## Computing Maximal 0.8-Quasi-Cliques with at Least 10 Vertices

Run quasi-clique program: ./run [input_data] [thread_num] [ratio] [min_size] [time_split_threshold]

In [ ]:
!Tthinker-main/app_qc/run input_data 5 0.8 10 5

Collect the result generated by different threads and post-process the result

In [ ]:
!cat output_* > result && rm output_* # get the "result" file
!Tthinker-main/maximal_check/quasiCliques result maximal_result # remove non-maximals

Read the edgelist

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pickle

fh=open("ca-GrQc.txt", 'rb')
#G=nx.read_adjlist(fh)
G=nx.read_edgelist(fh)
fh.close()

Get the top-2 quasi-clique in the result

In [ ]:
max_qc = set()
sec_qc = set()
max_qc_size = 0
sec_qc_size = 0
with open('maximal_result') as f:
    for line in f:
        cur_size = int(line.split()[0])
        if max_qc_size < cur_size:
            #replace second largest qc with largest qc
            sec_qc_size = max_qc_size
            sec_qc = max_qc
            # replace largest qc with current qc
            max_qc_size = cur_size
            max_qc = set(line.split()[1:])
        elif sec_qc_size < cur_size:
            # replace second largest qc with current qc
            sec_qc_size = cur_size
            sec_qc = set(line.split()[1:])
    
print(len(max_qc))
print(len(sec_qc))

Print the largest quasi-clique

In [ ]:
S1 = G.subgraph(max_qc)
nx.draw(S1)
plt.show()

Print the second largest quasi-clique

In [ ]:
S2 = G.subgraph(sec_qc)
#npos=nx.circular_layout(S2)
#nx.draw(S2, npos, with_labels = True, node_size = 30)
nx.draw(S2, with_labels = True)
plt.show()

In [ ]:
# The End